In [1]:
#extracting answer types
import json
import nltk
import os
import string
import pickle  #For caching of results
from dateutil.parser import parse
from time import ctime
import re
from nltk import StanfordPOSTagger
from nltk.tag.stanford import StanfordNERTagger

In [ ]:
nltk.download('wordnet')
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma
#print("Users",lemmatize('Users'))
#print("users",lemmatize('users'))
top_rules = {
            'where': 'LOCATION',
            'when': 'DATE',           
            'who': 'PERSON',     
            'how many': 'NUMBER',
            'how much': 'NUMBER', 
            'what scientist':'PERSON',
            'what athlete':'PERSON',
            'what date':'DATE',
            'what day':'DATE',
            'what year': 'DATE',
            'which company': 'ORGANIZATION'            

}

second_rules = {
             
            'stand for': 'DEFINITION',
            'mean': 'DEFINITION',
            'from': 'LOCATION',
            'country': 'LOCATION',
            'capital': 'LOCATION',
            'city': 'LOCATION',
            'person':'PERSON',
            'location': 'LOCATION',                       
            'long':'NUMBER',
            'high': 'NUMBER',
            'year': 'NUMBER',
            'decade': 'NUMBER',
            'time': 'NUMBER',
            'cost':'NUMBER',
            'population': 'NUMBER',
            'number':'NUMBER'
            

}


rule_names = []
for k,v in top_rules.items():
    rule_names.append(k)
#print("rule_names",rule_names)
second_rule_names = []
for k,v in second_rules.items():
    second_rule_names.append(k)
#print("second_rule_names",second_rule_names)
with open('testing.json','r') as test_file:
    test_data = json.load(test_file)
    
def get_all_questions(data):
    all_questions = []
    for question in data:
        current_question = question['question']
        all_questions.append(current_question)
    return all_questions
all_questions = get_all_questions(test_data)
#print("all_questions",all_questions)

def process_question(all_questions):
    translator = str.maketrans('', '', string.punctuation)
    all_processed_questions = []
    for current_question in all_questions:
        processed_question = []
        current_question = word_tokenize(current_question)
        for word in current_question:
            new_word = word.translate(translator)
            if new_word is not '':
                result_word = lemmatize(new_word.lower())
                processed_question.append(result_word)
        processed_question_str = " ".join(x for x in processed_question)
        #print("processed_question_str",processed_question_str)
        all_processed_questions.append(processed_question_str)
    return all_processed_questions
all_processed_questions = process_question(all_questions)
#print("all_processed_questions",all_processed_questions)


def tag_answer_type(all_processed_questions):
    questionNo = -1
    questions_data = []
    os.environ["STANFORD_MODELS"] = "/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/stanford-ner-2018-02-27"
    stanford_NER_tagger = StanfordNERTagger('/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/stanford-ner-2018-02-27/classifiers/english.muc.7class.distsim.crf.ser.gz','/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/stanford-ner-2018-02-27/stanford-ner.jar')
    stanford_POS_tagger = StanfordPOSTagger('/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/stanford-ner-2018-02-27/english-bidirectional-distsim.tagger','/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/stanford-ner-2018-02-27/stanford-postagger.jar')                                 

    for processed_question in all_processed_questions:
        questionNo+=1
        question_data = {}
        question_answer_type_dict = {}
        print("Processing Question: ", questionNo + 1, '/', len(all_processed_questions))
        NER_question = stanford_NER_tagger.tag_sents([processed_question.split()])
        NER_question_dict = {}
        for NER_item in NER_question:
            for NER_token in NER_item:
                NER_question_dict[NER_token[0]] = NER_token[1]
        question_data["NER_question"] = NER_question_dict
        #print("question_data--NER_question",question_data)
        POS_question = stanford_POS_tagger.tag_sents([processed_question.split()])
        POS_question_dict = {}
        for question in POS_question:
            for item in question:
                POS_question_dict[item[0]] = item[1]
        #print("POS_question",POS_question)
        #print("POS_question_dict",POS_question_dict)
        question_data["POS_question"] = POS_question_dict
        #print("question_data--POS_question",question_data)
                
                     
        if "what" in processed_question:
            #print("what in processed_question")
            tag = POS_question_dict.get("what",'NN')
            question_answer_type_dict[all_questions[questionNo]] = tag
            #print("question_answer_type_dict",question_answer_type_dict)
            
        elif "which" in processed_question:
            #print("which in processed_question")
            tag = POS_question_dict.get("which",'NN')
            question_answer_type_dict[all_questions[questionNo]] = tag
            
        elif "how" in processed_question:
            #print("how in processed_question")
            tag = POS_question_dict.get("how",'RB')
            question_answer_type_dict[all_questions[questionNo]] = tag
            
        else:
            for rule_name in second_rule_names:
                if rule_name in processed_question:
                    #print("second_rule_name in processed_question")
                    tag = second_rules.get(rule_name,"O")
                    #print("second_rule_name tag", tag)
                    question_answer_type_dict[all_questions[questionNo]] = tag
                
                else:
                    tag = second_rules.get(rule_name,"None")
                    question_answer_type_dict[all_questions[questionNo]] = tag
            
        for rule_name in rule_names: 
            if "what" in processed_question: #set "what" in the top priority
                tag = POS_question_dict.get("what",'NN')
            elif rule_name in processed_question:
                #print("rule_name in processed_question")
                tag = top_rules.get(rule_name,"O")
                question_answer_type_dict[all_questions[questionNo]] = tag  
                
        question_data["question_answer_type_dict"] = question_answer_type_dict
        questions_data.append(question_data)
                         
    return questions_data
                    
    
 
data = tag_answer_type(all_processed_questions)               
#print("data",data)
with open("QuestionLabel.json", 'w') as result_file:
    json.dump(data,result_file)
    print("Finished writing into result_file!")


[nltk_data] Downloading package wordnet to /Users/shimei/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/Users/shimei/anaconda3/lib/python3.6/site-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)
/Users/shimei/anaconda3/lib/python3.6/site-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


Processing Question:  1 / 3618
Processing Question:  2 / 3618
Processing Question:  3 / 3618
Processing Question:  4 / 3618
Processing Question:  5 / 3618
Processing Question:  6 / 3618
Processing Question:  7 / 3618
Processing Question:  8 / 3618
Processing Question:  9 / 3618
Processing Question:  10 / 3618
Processing Question:  11 / 3618
Processing Question:  12 / 3618
Processing Question:  13 / 3618
Processing Question:  14 / 3618
Processing Question:  15 / 3618
Processing Question:  16 / 3618
Processing Question:  17 / 3618
Processing Question:  18 / 3618
Processing Question:  19 / 3618
Processing Question:  20 / 3618
Processing Question:  21 / 3618
Processing Question:  22 / 3618
Processing Question:  23 / 3618
Processing Question:  24 / 3618
Processing Question:  25 / 3618
Processing Question:  26 / 3618
Processing Question:  27 / 3618
Processing Question:  28 / 3618
Processing Question:  29 / 3618
Processing Question:  30 / 3618
Processing Question:  31 / 3618
Processing Questi

Processing Question:  253 / 3618
Processing Question:  254 / 3618
Processing Question:  255 / 3618
Processing Question:  256 / 3618
Processing Question:  257 / 3618
Processing Question:  258 / 3618
Processing Question:  259 / 3618
Processing Question:  260 / 3618
Processing Question:  261 / 3618
Processing Question:  262 / 3618
Processing Question:  263 / 3618
Processing Question:  264 / 3618
Processing Question:  265 / 3618
Processing Question:  266 / 3618
Processing Question:  267 / 3618
Processing Question:  268 / 3618
Processing Question:  269 / 3618
Processing Question:  270 / 3618
Processing Question:  271 / 3618
Processing Question:  272 / 3618
Processing Question:  273 / 3618
Processing Question:  274 / 3618
Processing Question:  275 / 3618
Processing Question:  276 / 3618
Processing Question:  277 / 3618
Processing Question:  278 / 3618
Processing Question:  279 / 3618
Processing Question:  280 / 3618
Processing Question:  281 / 3618
Processing Question:  282 / 3618
Processing

Processing Question:  502 / 3618
Processing Question:  503 / 3618
Processing Question:  504 / 3618
Processing Question:  505 / 3618
Processing Question:  506 / 3618
Processing Question:  507 / 3618
Processing Question:  508 / 3618
Processing Question:  509 / 3618
Processing Question:  510 / 3618
Processing Question:  511 / 3618
Processing Question:  512 / 3618
Processing Question:  513 / 3618
Processing Question:  514 / 3618
Processing Question:  515 / 3618
Processing Question:  516 / 3618
Processing Question:  517 / 3618
Processing Question:  518 / 3618
Processing Question:  519 / 3618
Processing Question:  520 / 3618
Processing Question:  521 / 3618
Processing Question:  522 / 3618
Processing Question:  523 / 3618
Processing Question:  524 / 3618
Processing Question:  525 / 3618
Processing Question:  526 / 3618
Processing Question:  527 / 3618
Processing Question:  528 / 3618
Processing Question:  529 / 3618
Processing Question:  530 / 3618
Processing Question:  531 / 3618
Processing

...

Processing Question:  2205 / 3618
Processing Question:  2206 / 3618
Processing Question:  2207 / 3618
Processing Question:  2208 / 3618
Processing Question:  2209 / 3618
Processing Question:  2210 / 3618
Processing Question:  2211 / 3618
Processing Question:  2212 / 3618
Processing Question:  2213 / 3618
Processing Question:  2214 / 3618
Processing Question:  2215 / 3618
Processing Question:  2216 / 3618
Processing Question:  2217 / 3618
Processing Question:  2218 / 3618
Processing Question:  2219 / 3618
Processing Question:  2220 / 3618
Processing Question:  2221 / 3618
Processing Question:  2222 / 3618
Processing Question:  2223 / 3618
Processing Question:  2224 / 3618
Processing Question:  2225 / 3618
Processing Question:  2226 / 3618
Processing Question:  2227 / 3618
Processing Question:  2228 / 3618
Processing Question:  2229 / 3618
Processing Question:  2230 / 3618
Processing Question:  2231 / 3618
Processing Question:  2232 / 3618
Processing Question:  2233 / 3618
Processing Que

Processing Question:  2446 / 3618
Processing Question:  2447 / 3618
Processing Question:  2448 / 3618
Processing Question:  2449 / 3618
Processing Question:  2450 / 3618
Processing Question:  2451 / 3618
Processing Question:  2452 / 3618
Processing Question:  2453 / 3618
Processing Question:  2454 / 3618
Processing Question:  2455 / 3618
Processing Question:  2456 / 3618
Processing Question:  2457 / 3618
Processing Question:  2458 / 3618
Processing Question:  2459 / 3618
Processing Question:  2460 / 3618
Processing Question:  2461 / 3618
Processing Question:  2462 / 3618
Processing Question:  2463 / 3618
Processing Question:  2464 / 3618
Processing Question:  2465 / 3618
Processing Question:  2466 / 3618
Processing Question:  2467 / 3618
Processing Question:  2468 / 3618
Processing Question:  2469 / 3618
Processing Question:  2470 / 3618
Processing Question:  2471 / 3618
Processing Question:  2472 / 3618
Processing Question:  2473 / 3618
Processing Question:  2474 / 3618
Processing Que

Processing Question:  2687 / 3618
Processing Question:  2688 / 3618
Processing Question:  2689 / 3618
Processing Question:  2690 / 3618
Processing Question:  2691 / 3618
Processing Question:  2692 / 3618
Processing Question:  2693 / 3618
Processing Question:  2694 / 3618
Processing Question:  2695 / 3618
Processing Question:  2696 / 3618
Processing Question:  2697 / 3618
Processing Question:  2698 / 3618
Processing Question:  2699 / 3618
Processing Question:  2700 / 3618
Processing Question:  2701 / 3618
Processing Question:  2702 / 3618
Processing Question:  2703 / 3618
Processing Question:  2704 / 3618
Processing Question:  2705 / 3618
Processing Question:  2706 / 3618
Processing Question:  2707 / 3618
Processing Question:  2708 / 3618
Processing Question:  2709 / 3618
Processing Question:  2710 / 3618
Processing Question:  2711 / 3618
Processing Question:  2712 / 3618
Processing Question:  2713 / 3618
Processing Question:  2714 / 3618
Processing Question:  2715 / 3618
Processing Que

Processing Question:  2928 / 3618
Processing Question:  2929 / 3618
Processing Question:  2930 / 3618
Processing Question:  2931 / 3618
Processing Question:  2932 / 3618
Processing Question:  2933 / 3618
Processing Question:  2934 / 3618
Processing Question:  2935 / 3618
Processing Question:  2936 / 3618
Processing Question:  2937 / 3618
Processing Question:  2938 / 3618
Processing Question:  2939 / 3618
Processing Question:  2940 / 3618
Processing Question:  2941 / 3618
Processing Question:  2942 / 3618
Processing Question:  2943 / 3618
Processing Question:  2944 / 3618
Processing Question:  2945 / 3618
Processing Question:  2946 / 3618
Processing Question:  2947 / 3618
Processing Question:  2948 / 3618
Processing Question:  2949 / 3618
Processing Question:  2950 / 3618
Processing Question:  2951 / 3618
Processing Question:  2952 / 3618
Processing Question:  2953 / 3618
Processing Question:  2954 / 3618
Processing Question:  2955 / 3618
Processing Question:  2956 / 3618
Processing Que

Processing Question:  3169 / 3618
Processing Question:  3170 / 3618
Processing Question:  3171 / 3618
Processing Question:  3172 / 3618
Processing Question:  3173 / 3618
Processing Question:  3174 / 3618
Processing Question:  3175 / 3618
Processing Question:  3176 / 3618
Processing Question:  3177 / 3618
Processing Question:  3178 / 3618
Processing Question:  3179 / 3618
Processing Question:  3180 / 3618
Processing Question:  3181 / 3618
Processing Question:  3182 / 3618
Processing Question:  3183 / 3618
Processing Question:  3184 / 3618
Processing Question:  3185 / 3618
Processing Question:  3186 / 3618
Processing Question:  3187 / 3618
Processing Question:  3188 / 3618
Processing Question:  3189 / 3618
Processing Question:  3190 / 3618
Processing Question:  3191 / 3618
Processing Question:  3192 / 3618
Processing Question:  3193 / 3618
Processing Question:  3194 / 3618
Processing Question:  3195 / 3618
Processing Question:  3196 / 3618
Processing Question:  3197 / 3618
Processing Que